# 加载数据

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [30]:
# 设置随机种子
SEED=222
np.random.seed(SEED)

In [31]:
df=pd.read_csv('files/data/Python88/input.csv')

In [34]:
def get_train_test(test_size=0.95):

    y = 1 * (df.cand_pty_affiliation == "REP")

    X = df.drop(["cand_pty_affiliation"], axis=1)

    X = pd.get_dummies(X, sparse=True)

    X.drop(X.columns[X.std() == 0], axis=1, inplace=True)
    X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=test_size, random_state=SEED)
    print(X_train)
    return train_test_split(X, y, test_size=test_size, random_state=SEED)

X_train, X_test, y_train, y_test = get_train_test()
print("\nExample data:")

df.head()


TypeError: values must be SparseArray

In [ ]:
%matplotlib inline
df['cand_pty_affiliation'].value_counts().plot(kind='bar')

从上图可以看出，75% 的捐款是给民主党的

我们使用 ROC-AUC 来评估模型性能。如果你之前没用过该指标，随机猜测可以是 0.5 分，完美的召回率和精确率是 1.0。

## 模型集成

想象一下你在玩常识问答游戏。一个人玩时，可能会有一些题你完全不了解。如果我们想获得高分，就需要组建一个团队来覆盖所有相关主题。这就是集成的基本概念：结合多个模型的预测，对特异性误差取平均，从而获得更好的整体预测结果。

一个重要问题是如何结合预测。以常识问答游戏为例，我们很容易想象到团队成员可能会使用多数投票的方式确定选择哪个答案。机器学习的分类问题也是一样：作出最常见的类别标签预测相当于多数投票规则。但是也有很多其他方式可以结合预测，通常我们会使用一个模型来学习如何最好地结合预测结果。

![](https://www.dataquest.io/blog/content/images/2018/01/ensemble_network.png)基础集成的结构。输入输送至一系列模型中，元学习器将多个模型的预测结果结合起来。

https://www.jiqizhixin.com/articles/2018-01-14-8

## 决策树

我们用一个简单的可解释性模型来解释集成：使用 if-then 规则的决策树。决策树越深，可以捕捉的模式就越复杂，不过也更有可能出现过拟合。因此，我们需要另一种方式来构建决策树的复杂模型，而不同决策树的集成就是这样一种方式。

In [ ]:
import pydotplus
from IPython.display import Image
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier,export_graphviz
def print_graph(clf, feature_names):

    """Print decision tree."""

    graph = export_graphviz(

        clf,

        label="root",

        proportion=True,

        impurity=False, 

        out_file=None, 

        feature_names=feature_names,

        class_names={0: "D", 1: "R"},

        filled=True,

        rounded=True

    )

    graph = pydotplus.graph_from_dot_data(graph)  

    return Image(graph.create_png())

在训练数据上用决策树拟合一个节点（决策规则），查看它在测试集上的性能：

In [ ]:
t1 = DecisionTreeClassifier(max_depth=1, random_state=SEED)

t1.fit(X_train, y_train)

p = t1.predict_proba(X_test)[:, 1]


print("Decision tree ROC-AUC score: %.3f" % roc_auc_score(y_test, p))

print_graph(t1, X_train.columns)

每个叶节点记录它们在训练样本中的比例、类别分布和类别标签预测。我们的决策树根据捐款金额是否超过 101.5 进行预测：它竟然作出了同样的预测！鉴于 75% 的捐款都给了民主党，这个结果并不令人惊讶。但是这没有充分利用我们已有的数据，下面我们使用三层决策规则，看看会得到什么

In [ ]:
t2 = DecisionTreeClassifier(max_depth=3, random_state=SEED)

t2.fit(X_train, y_train)

p = t2.predict_proba(X_test)[:, 1]


print("Decision tree ROC-AUC score: %.3f" % roc_auc_score(y_test, p))

print_graph(t2, X_train.columns)

该模型并不比简单的决策树好太多：预测到的共和党捐款金额比例只有 5%，远远低于 25%。仔细观察会发现该决策树使用了很多不确定的分割规则（splitting rule）。观察结果中高达 47.3% 的结果在最左边的叶节点中，而 35.9% 在右二的叶节点中。因此大量叶节点没有关联。使该模型更深只会导致过拟合。

在深度固定的情况下，决策树可以通过增加「宽度」的方式来增加复杂度，即创建多个决策树，并将其连接起来。也就是决策树的集成。想了解这个集成模型为什么会起作用，先要考虑我们如何让决策树探索出比上层树更多的其他模式。最简单的解决方案就是删除树中较早出现的特征。假如我们删除了转账金额特征（transaction_amt），树的根节点，则新的决策树如下：

In [ ]:
drop = ["transaction_amt"]
# X_train

# xtrain_slim = X_train.drop(drop, 1)

# xtest_slim = X_test.drop(drop, 1)


# t3 = DecisionTreeClassifier(max_depth=3, random_state=SEED)

# t3.fit(xtrain_slim, y_train)

# p = t3.predict_proba(xtest_slim)[:, 1]



# print("Decision tree ROC-AUC score: %.3f" % roc_auc_score(y_test, p))

# print_graph(t3, xtrain_slim.columns)